# Phase 2 — Feature Engineering
**Inputs:** `X_train.csv`, `y_train.csv` (from Phase 1)  
**Outputs:** `X_train_fe.csv`, `y_train_fe.csv`  
**Constraints:** No SMOTE · No modeling · No future data · No test-set touch

---

## Leakage Prevention Strategy

Every feature in this notebook is computed using **only past information** at the time of each transaction:

| Risk | Mitigation |
|---|---|
| Using future transactions in rolling windows | `.shift(1)` before `.rolling()` so window sits entirely in the past |
| Amount normalisation using test statistics | `mean` and `std` computed on **train only**, stored as constants |
| Fraud-rate features leaking current label | `shift(1)` applied to `Class` before rolling, excluding the current row |
| NaN fill introducing future info | Rows with NaN from cold-start dropped entirely (no forward/backward fill) |

## Load Data

In [8]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# ── Load train split from Phase 1 ─────────────────────────────────────────────
X_train = pd.read_csv("data/processed/X_train.csv")
y_train = pd.read_csv("data/processed/y_train.csv")

# Combine for easier rolling computation; keep original row order (Time-sorted from Phase 1)
train = X_train.copy()
train["Class"] = y_train["Class"].values

print(f"Loaded  X_train: {X_train.shape}  |  y_train: {y_train.shape}")
print(f"Columns: {list(X_train.columns)}")
print(f"Time range: {train['Time'].min():.0f}s → {train['Time'].max():.0f}s")
print(f"Fraud rate: {train['Class'].mean() * 100:.4f}%")

Loaded  X_train: (227845, 30)  |  y_train: (227845, 1)
Columns: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
Time range: 0s → 145247s
Fraud rate: 0.1830%


## Step 1 — Basic Transformations

**`log_amount`** compresses the heavy right-skew of transaction amounts.  
**`amount_zscore_global`** standardises amount using *train-only* statistics — the stored `mean_train` / `std_train` will be reused later when transforming the test set, ensuring no test information bleeds into training.

In [9]:
# ── Ensure chronological order ────────────────────────────────────────────────
train = train.sort_values("Time").reset_index(drop=True)

# ── log_amount ─────────────────────────────────────────────────────────────────
# log(1 + x) keeps 0-valued transactions valid and compresses right-skew
train["log_amount"] = np.log1p(train["Amount"])

# ── amount_zscore_global ───────────────────────────────────────────────────────
# Statistics computed on TRAIN ONLY — must not use test set here
mean_train = train["Amount"].mean()
std_train  = train["Amount"].std(ddof=0)          # population std for consistency

train["amount_zscore_global"] = (train["Amount"] - mean_train) / std_train

# Persist scaling constants so they can be applied to test set without leakage
scaling_params = {"mean_amount": mean_train, "std_amount": std_train}
pd.DataFrame([scaling_params]).to_csv("data/processed/train_scaling_params.csv", index=False)

print(f"Amount  →  mean={mean_train:.4f}  std={std_train:.4f}")
print(f"log_amount       : min={train['log_amount'].min():.3f}  max={train['log_amount'].max():.3f}")
print(f"amount_zscore_global: min={train['amount_zscore_global'].min():.3f}  max={train['amount_zscore_global'].max():.3f}")
print(f"\nScaling params saved to data/processed/train_scaling_params.csv")

Amount  →  mean=90.8249  std=250.5032
log_amount       : min=0.000  max=9.886
amount_zscore_global: min=-0.363  max=78.106

Scaling params saved to data/processed/train_scaling_params.csv


## Step 2 — Rolling Behavioral Features (Time-Safe)

Because the dataset has no individual card/user ID, all transactions are treated as a single chronological stream ordered by `Time`.

**Leakage prevention:** `.shift(1)` is applied *before* `.rolling()`. This means the rolling window for row *i* covers rows *0 … i-1* — the current transaction is never included in its own feature computation.

| Feature | Window | Meaning |
|---|---|---|
| `rolling_mean_amount` | 100 txns | Typical recent transaction size |
| `rolling_std_amount` | 100 txns | Recent volatility in amounts |
| `time_diff` | 1 txn | Gap (seconds) since last transaction |

In [10]:
WINDOW_AMOUNT = 100

# ── Shift Amount by 1 so row i only sees rows 0…i-1 ──────────────────────────
amount_shifted = train["Amount"].shift(1)

# ── Rolling mean and std of Amount (past window only) ────────────────────────
train["rolling_mean_amount"] = (
    amount_shifted
    .rolling(window=WINDOW_AMOUNT, min_periods=1)
    .mean()
)
train["rolling_std_amount"] = (
    amount_shifted
    .rolling(window=WINDOW_AMOUNT, min_periods=2)   # std needs ≥2 points
    .std(ddof=1)
)

# ── Time difference between consecutive transactions ─────────────────────────
# shift(1) gives the previous row's Time; difference = elapsed seconds since last txn
train["time_diff"] = train["Time"] - train["Time"].shift(1)
# First row has no predecessor — leave as NaN (will be dropped in Step 5)

print("Rolling behavioral features added:")
print(f"  rolling_mean_amount  — {train['rolling_mean_amount'].notna().sum():,} non-null")
print(f"  rolling_std_amount   — {train['rolling_std_amount'].notna().sum():,} non-null")
print(f"  time_diff            — {train['time_diff'].notna().sum():,} non-null")
print(f"\nSample (rows 98–103):")
print(train[["Time", "Amount", "rolling_mean_amount", "rolling_std_amount", "time_diff"]].iloc[98:104].to_string(index=True))

Rolling behavioral features added:
  rolling_mean_amount  — 227,844 non-null
  rolling_std_amount   — 227,843 non-null
  time_diff            — 227,844 non-null

Sample (rows 98–103):
     Time  Amount  rolling_mean_amount  rolling_std_amount  time_diff
98   67.0   28.28            65.635306          186.355543        0.0
99   68.0   15.99            65.257980          185.440320        1.0
100  68.0    2.69            64.765300          184.567145        0.0
101  68.0   22.36            63.296000          184.469625        0.0
102  69.0    9.47            63.492700          184.414827        1.0
103  69.0    0.76            59.800800          181.717369        0.0


## Step 3 — Amount Deviation Features

These capture how unusual the *current* transaction amount is relative to the recent stream.

- **`amount_deviation`** — raw distance from the rolling mean.  
- **`amount_zscore_rolling`** — normalised deviation; `np.where` guards against division by zero when `rolling_std_amount` is 0 or NaN.

In [11]:
# ── amount_deviation: how far current Amount is from the rolling mean ─────────
train["amount_deviation"] = train["Amount"] - train["rolling_mean_amount"]

# ── amount_zscore_rolling: deviation normalised by rolling std ────────────────
# Safe division: if rolling_std is 0 or NaN → result is NaN (will be dropped later)
std_safe = train["rolling_std_amount"].replace(0, np.nan)   # treat 0 std as undefined
train["amount_zscore_rolling"] = train["amount_deviation"] / std_safe

print("Amount deviation features added:")
print(f"  amount_deviation      — {train['amount_deviation'].notna().sum():,} non-null")
print(f"  amount_zscore_rolling — {train['amount_zscore_rolling'].notna().sum():,} non-null")
print(f"\nDescriptive stats:")
print(train[["amount_deviation", "amount_zscore_rolling"]].describe().round(4))

Amount deviation features added:
  amount_deviation      — 227,844 non-null
  amount_zscore_rolling — 227,843 non-null

Descriptive stats:
       amount_deviation  amount_zscore_rolling
count       227844.0000            227843.0000
mean            -0.0035                 0.0518
std            251.0174                 1.5976
min           -295.4574                -2.4175
25%            -83.2858                -0.4253
50%            -53.6495                -0.2963
75%             -5.6945                -0.0335
max          19557.1228               155.6727


## Step 4 — Fraud Momentum Features

These features simulate **fraud bursts** — periods of elevated fraud activity.

**Critical leakage point:** The `Class` label of row *i* must not contribute to its own feature. `.shift(1)` moves the label series back by one position, so the rolling window for row *i* covers labels of rows *0 … i-1* only.

| Feature | Window | Meaning |
|---|---|---|
| `rolling_fraud_rate_500` | 500 txns | Share of recent txns flagged as fraud |
| `rolling_fraud_count_500` | 500 txns | Absolute count of recent fraud txns |

In [12]:
WINDOW_FRAUD = 500

# ── Shift Class by 1 to exclude the current row's label from its own window ───
class_shifted = train["Class"].shift(1)

# ── Rolling fraud count (sum of past WINDOW_FRAUD labels) ────────────────────
train["rolling_fraud_count_500"] = (
    class_shifted
    .rolling(window=WINDOW_FRAUD, min_periods=1)
    .sum()
)

# ── Rolling fraud rate (mean of past WINDOW_FRAUD labels) ────────────────────
train["rolling_fraud_rate_500"] = (
    class_shifted
    .rolling(window=WINDOW_FRAUD, min_periods=1)
    .mean()
)

print(f"Fraud momentum features (window={WINDOW_FRAUD}) added:")
print(f"  rolling_fraud_count_500 — {train['rolling_fraud_count_500'].notna().sum():,} non-null")
print(f"  rolling_fraud_rate_500  — {train['rolling_fraud_rate_500'].notna().sum():,} non-null")

print(f"\nFraud rate feature stats:")
print(train["rolling_fraud_rate_500"].describe().round(6))

# Quick sanity check: fraud in full training set vs rolling feature range
print(f"\nGlobal fraud rate in train         : {train['Class'].mean():.6f}")
print(f"Mean of rolling_fraud_rate_500     : {train['rolling_fraud_rate_500'].mean():.6f}")

Fraud momentum features (window=500) added:
  rolling_fraud_count_500 — 227,844 non-null
  rolling_fraud_rate_500  — 227,844 non-null

Fraud rate feature stats:
count    227844.000000
mean          0.001830
std           0.003992
min           0.000000
25%           0.000000
50%           0.000000
75%           0.002000
max           0.056000
Name: rolling_fraud_rate_500, dtype: float64

Global fraud rate in train         : 0.001830
Mean of rolling_fraud_rate_500     : 0.001830


## Step 5 — Clean Dataset & Save

Rolling windows produce NaN values at the start of the series (cold-start). These rows are dropped **entirely** — no forward/backward fill, no imputation with future values.

The `Class` column is separated into `y_train_fe` and the engineered feature matrix is saved as `X_train_fe`.

In [13]:
import os

# ── All engineered feature columns ────────────────────────────────────────────
ENGINEERED_COLS = [
    "log_amount",
    "amount_zscore_global",
    "rolling_mean_amount",
    "rolling_std_amount",
    "time_diff",
    "amount_deviation",
    "amount_zscore_rolling",
    "rolling_fraud_count_500",
    "rolling_fraud_rate_500",
]

TARGET = "Class"

# ── Feature matrix: original features + engineered features ──────────────────
original_features = [c for c in X_train.columns]          # from Phase 1 (no Class)
all_feature_cols  = original_features + ENGINEERED_COLS

feature_df = train[all_feature_cols + [TARGET]].copy()

# ── Drop NaN rows — cold-start artefacts from rolling windows ────────────────
rows_before = len(feature_df)
feature_df.dropna(inplace=True)
rows_after  = len(feature_df)

print(f"Rows before NaN drop : {rows_before:,}")
print(f"Rows dropped (NaN)   : {rows_before - rows_after:,}")
print(f"Rows after  NaN drop : {rows_after:,}")
print(f"Fraud rate after drop: {feature_df[TARGET].mean() * 100:.4f}%")

# ── Separate X and y ──────────────────────────────────────────────────────────
X_train_fe = feature_df[all_feature_cols].reset_index(drop=True)
y_train_fe = feature_df[[TARGET]].reset_index(drop=True)

# ── Save ──────────────────────────────────────────────────────────────────────
PROCESSED = "data/processed"
os.makedirs(PROCESSED, exist_ok=True)

X_train_fe.to_csv(f"{PROCESSED}/X_train_fe.csv", index=False)
y_train_fe.to_csv(f"{PROCESSED}/y_train_fe.csv", index=False)

print(f"\nSaved:")
for fname, obj in [("X_train_fe.csv", X_train_fe), ("y_train_fe.csv", y_train_fe)]:
    print(f"  {fname:20s} → {obj.shape[0]:,} rows × {obj.shape[1]} col(s)")

print(f"\nEngineered feature columns ({len(ENGINEERED_COLS)}):")
for col in ENGINEERED_COLS:
    print(f"  {col}")

Rows before NaN drop : 227,845
Rows dropped (NaN)   : 2
Rows after  NaN drop : 227,843
Fraud rate after drop: 0.1830%

Saved:
  X_train_fe.csv       → 227,843 rows × 39 col(s)
  y_train_fe.csv       → 227,843 rows × 1 col(s)

Engineered feature columns (9):
  log_amount
  amount_zscore_global
  rolling_mean_amount
  rolling_std_amount
  time_diff
  amount_deviation
  amount_zscore_rolling
  rolling_fraud_count_500
  rolling_fraud_rate_500


## Feature Summary & Leakage Audit

In [14]:
print("=" * 60)
print("FEATURE ENGINEERING SUMMARY")
print("=" * 60)

summary = {
    "log_amount":               "log(1 + Amount)                   — no future info",
    "amount_zscore_global":     "(Amount − train_mean) / train_std — train stats only",
    "rolling_mean_amount":      "shift(1) + rolling(100).mean()    — past only",
    "rolling_std_amount":       "shift(1) + rolling(100).std()     — past only",
    "time_diff":                "Time − Time.shift(1)              — previous row only",
    "amount_deviation":         "Amount − rolling_mean_amount      — past only",
    "amount_zscore_rolling":    "amount_deviation / rolling_std    — past only",
    "rolling_fraud_count_500":  "shift(1) + rolling(500).sum()     — past labels only",
    "rolling_fraud_rate_500":   "shift(1) + rolling(500).mean()    — past labels only",
}

for feat, desc in summary.items():
    null_pct = X_train_fe[feat].isna().mean() * 100
    print(f"  {feat:<30s}  NaN%={null_pct:.2f}   {desc}")

print()
print(f"Final X_train_fe shape : {X_train_fe.shape}")
print(f"Final y_train_fe shape : {y_train_fe.shape}")
print(f"Fraud rate             : {y_train_fe['Class'].mean() * 100:.4f}%")
print()
print("Leakage checks passed:")
print("  ✓ No test-set data used at any point")
print("  ✓ All rolling windows apply shift(1) before rolling")
print("  ✓ Fraud-rate features use shifted Class labels")
print("  ✓ Global normalisation uses train-only mean/std")
print("  ✓ NaN rows dropped (not filled with future values)")
print("=" * 60)
print("Phase 2 complete.")

FEATURE ENGINEERING SUMMARY
  log_amount                      NaN%=0.00   log(1 + Amount)                   — no future info
  amount_zscore_global            NaN%=0.00   (Amount − train_mean) / train_std — train stats only
  rolling_mean_amount             NaN%=0.00   shift(1) + rolling(100).mean()    — past only
  rolling_std_amount              NaN%=0.00   shift(1) + rolling(100).std()     — past only
  time_diff                       NaN%=0.00   Time − Time.shift(1)              — previous row only
  amount_deviation                NaN%=0.00   Amount − rolling_mean_amount      — past only
  amount_zscore_rolling           NaN%=0.00   amount_deviation / rolling_std    — past only
  rolling_fraud_count_500         NaN%=0.00   shift(1) + rolling(500).sum()     — past labels only
  rolling_fraud_rate_500          NaN%=0.00   shift(1) + rolling(500).mean()    — past labels only

Final X_train_fe shape : (227843, 39)
Final y_train_fe shape : (227843, 1)
Fraud rate             : 0.1830%

